In [1]:
# Importing Libraries

In [6]:
import pandas as pd
import numpy as np

In [7]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [8]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [9]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [25]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [27]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [21]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [15]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [17]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [18]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [28]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [29]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [30]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

2023-01-05 16:02:07.734649: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-05 16:02:07.735500: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [32]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30


2023-01-05 16:02:15.983963: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-05 16:02:16.501545: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-05 16:02:16.678164: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-05 16:02:17.394501: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


460/460 [==============================] - ETA: 0s - loss: 1.1574 - accuracy: 0.5311

2023-01-05 16:02:34.955457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-05 16:02:34.991466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


460/460 [==============================] - 24s 48ms/step - loss: 1.1574 - accuracy: 0.5311 - val_loss: 1.0551 - val_accuracy: 0.5843
Epoch 2/30
460/460 [==============================] - 20s 44ms/step - loss: 0.8118 - accuracy: 0.6806 - val_loss: 0.7874 - val_accuracy: 0.6882
Epoch 3/30
460/460 [==============================] - 20s 43ms/step - loss: 0.5680 - accuracy: 0.7926 - val_loss: 0.7801 - val_accuracy: 0.7682
Epoch 4/30
460/460 [==============================] - 20s 44ms/step - loss: 0.3858 - accuracy: 0.8841 - val_loss: 0.6659 - val_accuracy: 0.7937
Epoch 5/30
460/460 [==============================] - 20s 44ms/step - loss: 0.2779 - accuracy: 0.9159 - val_loss: 0.4930 - val_accuracy: 0.8565
Epoch 6/30
460/460 [==============================] - 20s 44ms/step - loss: 0.2470 - accuracy: 0.9218 - val_loss: 0.4897 - val_accuracy: 0.8463
Epoch 7/30
460/460 [==============================] - 20s 44ms/step - loss: 0.2055 - accuracy: 0.9308 - val_loss: 0.4180 - val_accuracy: 0.8724
Epo

In [33]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

2023-01-05 16:12:33.645407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-05 16:12:33.673548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


93/93 [==============================] - 2s 21ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      261       225        0                   0   
STANDING                 0       29       500        3                   0   
WALKING                  0        0         0      465                  11   
WALKING_DOWNSTAIRS       0        0         0       15                 394   
WALKING_UPSTAIRS         0        0         0       25                   6   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            5  
STANDING                           0  
WALKING                           20  
WALKING_DOWNSTAIRS                11  
WALKING_UPSTAIRS                 440  


In [34]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 2s 21ms/step - loss: 0.7051 - accuracy: 0.8721


In [35]:
score

[0.7050734162330627, 0.8720732927322388]

- With a simple 2 layer architecture we got 87.20% accuracy and a loss of 0.70
- We can further imporve the performace with Hyperparameter tuning